**Simulation groep 2 - RF detection**

In [2]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from IPython.display import display

In [3]:
def rssi_from_distance(d, p_tx_dbm= -30.0, n=2.0, noise_db=0.0):
    d = np.maximum(d, 1e-6)
    mean = p_tx_dbm - 10.0 * n * np.log10(d)
    noise = np.random.normal(0.0, noise_db, size=np.shape(d))
    return mean + noise

In [4]:
def distance_from_rssi(rssi_dbm, p_tx_dbm=-30.0, n=2.0):
    return 10.0 ** ((p_tx_dbm - rssi_dbm) / (10.0 * n))

In [5]:
def multilateration_linear(positions, distances):
    positions = np.asarray(positions, dtype=float)
    distances = np.asarray(distances, dtype=float)
    N = positions.shape[0]
    if N < 3:
        raise ValueError("Minimaal 3 referenties nodig")
    x1, y1 = positions[0]
    d1 = distances[0]
    A = []
    b = []
    for i in range(1, N):
        xi, yi = positions[i]
        di = distances[i]
        Ai = [2*(xi - x1), 2*(yi - y1)]
        bi = (xi**2 + yi**2 - di**2) - (x1**2 + y1**2 - d1**2)
        A.append(Ai)
        b.append(bi)
    A = np.array(A)
    b = np.array(b)
    sol, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
    return sol

In [6]:
def simulate_and_plot_once(pis, hotspot, p_tx_dbm=-30.0, path_loss_n=2.0, noise_db=2.0, fname=None):
    true_distances = np.linalg.norm(pis - hotspot, axis=1)
    measured_rssi = rssi_from_distance(true_distances, p_tx_dbm=p_tx_dbm, n=path_loss_n, noise_db=noise_db)
    estimated_distances = distance_from_rssi(measured_rssi, p_tx_dbm=p_tx_dbm, n=path_loss_n)
    est_pos = multilateration_linear(pis, estimated_distances)
    est_error = np.linalg.norm(est_pos - hotspot)

    df = pd.DataFrame({
        "Pi": ["Pi1","Pi2","Pi3"],
        "x (m)": np.round(pis[:,0],3),
        "y (m)": np.round(pis[:,1],3),
        "true_distance (m)": np.round(true_distances,3),
        "measured_RSSI (dBm)": np.round(measured_rssi,3),
        "estimated_distance (m)": np.round(estimated_distances,3)
    })

    try:
        from caas_jupyter_tools import display_dataframe_to_user
        display_dataframe_to_user("RF measurements", df)
    except Exception:
        display(df)

    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_aspect('equal', adjustable='box')
    ax.scatter(pis[:,0], pis[:,1], s=80)
    for i, (xx,yy) in enumerate(pis):
        ax.annotate(f"Pi{i+1}\n({xx:.1f},{yy:.1f})", (xx,yy), textcoords="offset points", xytext=(4,4))
    ax.scatter([hotspot[0]], [hotspot[1]], marker='x', s=100)
    ax.annotate(f"Hotspot true\n({hotspot[0]:.1f},{hotspot[1]:.1f})", (hotspot[0],hotspot[1]), textcoords="offset points", xytext=(4,-12))
    ax.scatter([est_pos[0]], [est_pos[1]], marker='o', s=80)
    ax.annotate(f"Est. ({est_pos[0]:.2f},{est_pos[1]:.2f})\n err={est_error:.2f} m", (est_pos[0], est_pos[1]), textcoords="offset points", xytext=(6,6))
    theta = np.linspace(0, 2*np.pi, 200)
    for i in range(3):
        r = estimated_distances[i]
        cx, cy = pis[i]
        ax.plot(cx + r*np.cos(theta), cy + r*np.sin(theta))
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_title("Simulatie: RSSI-based trilateratie (single run)")
    ax.grid(True)
    if fname:
        fig.savefig(fname, dpi=150)
        print(f"Grafiek opgeslagen naar: {fname}")
    plt.show()

    print(f"Transmit power P_tx = {p_tx_dbm} dBm, path-loss exponent n = {path_loss_n}, noise std = {noise_db} dB")
    print(f"True hotspot position: ({hotspot[0]:.3f}, {hotspot[1]:.3f})")
    print(f"Estimated position: ({est_pos[0]:.3f}, {est_pos[1]:.3f})  —  fout: {est_error:.3f} m")


In [12]:
try:
    import ipywidgets as widgets
    from ipywidgets import FloatSlider, interact, VBox, HBox, Layout
    # build interactive UI (same as earlier)
    layout = Layout(width='320px')
    pi_slider_x1 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi1 x', layout=layout)
    pi_slider_y1 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi1 y', layout=layout)
    pi_slider_x2 = FloatSlider(value=10.0, min=-20.0, max=20.0, step=0.5, description='Pi2 x', layout=layout)
    pi_slider_y2 = FloatSlider(value=0.0, min=-20.0, max=20.0, step=0.5, description='Pi2 y', layout=layout)
    pi_slider_x3 = FloatSlider(value=5.0, min=-20.0, max=20.0, step=0.5, description='Pi3 x', layout=layout)
    pi_slider_y3 = FloatSlider(value=8.0, min=-20.0, max=20.0, step=0.5, description='Pi3 y', layout=layout)

    hot_x = FloatSlider(value=4.0, min=-20.0, max=20.0, step=0.5, description='Hotspot x', layout=layout)
    hot_y = FloatSlider(value=3.0, min=-20.0, max=20.0, step=0.5, description='Hotspot y', layout=layout)

    p_tx_slider = FloatSlider(value=-30.0, min=-90.0, max=0.0, step=1.0, description='P_tx (dBm)', layout=layout)
    n_slider = FloatSlider(value=2.0, min=1.0, max=4.5, step=0.1, description='path-loss n', layout=layout)
    noise_slider = FloatSlider(value=2.0, min=0.0, max=12.0, step=0.1, description='noise σ (dB)', layout=layout)

    def interactive_sim(x1,y1,x2,y2,x3,y3,hx,hy,p_tx_dbm,path_loss_n,noise_db):
        pis = np.array([[x1,y1],[x2,y2],[x3,y3]])
        hotspot = np.array([hx,hy])
        simulate_and_plot_once(pis, hotspot, p_tx_dbm=p_tx_dbm, path_loss_n=path_loss_n, noise_db=noise_db)

    ui = VBox([HBox([VBox([pi_slider_x1, pi_slider_y1, pi_slider_x2, pi_slider_y2, pi_slider_x3, pi_slider_y3]),
                     VBox([hot_x, hot_y, p_tx_slider, n_slider, noise_slider])])])
    out = widgets.interactive_output(interactive_sim, {
        'x1': pi_slider_x1, 'y1': pi_slider_y1,
        'x2': pi_slider_x2, 'y2': pi_slider_y2,
        'x3': pi_slider_x3, 'y3': pi_slider_y3,
        'hx': hot_x, 'hy': hot_y,
        'p_tx_dbm': p_tx_slider, 'path_loss_n': n_slider, 'noise_db': noise_slider
    })
    display(ui, out)
    print("ipywidgets beschikbaar — interactieve sliders getoond.")

except Exception as e:
    print("ipywidgets niet beschikbaar in deze omgeving — voer een single-run voorbeeld uit.")
    # default example
    pis = np.array([[0.0,0.0],[10.0,0.0],[5.0,8.0]])
    hotspot = np.array([4.0,3.0])
    out_file = "/mnt/data/rf_simulation_example.png"
    simulate_and_plot_once(pis, hotspot, p_tx_dbm=-30.0, path_loss_n=2.0, noise_db=2.0, fname=out_file)
    print(f"[Download de afbeelding](/mnt/data/rf_simulation_example.png)")

Output()

ipywidgets beschikbaar — interactieve sliders getoond.


In [15]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)